### New Agent Framework with Langchain Release v0.1.0

<p>
<img src="https://blog.langchain.dev/content/images/size/w1248/format/webp/2024/01/V0.1.0_Export--1-.png" 
      width="35%" height="auto"
      style="display: block; margin: 0 auto" />

[Update notes](https://blog.langchain.dev/langchain-v0-1-0/)

# Agents

In [17]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WebBaseLoader, Docx2txtLoader
from langchain_community.vectorstores import chroma
from langchain_openai import OpenAIEmbeddings

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""


### Tools are the mini-engine which make the Agent work

Load the Langchain built-in tool Tavily for live online search

In [6]:
online_search = TavilySearchResults()

In [7]:
online_search.invoke("Cuanto quedo el Boys contra el Cristal?")

[{'url': 'https://www.alaskacommons.com/sporting-cristal-vs-sport-boys-live-rimenses-win-3-0-in-liga-1-apertura-2024/123235/',
  'content': 'Sporting Cristal vs Sport Boys LIVE: ‘Rimenses’ win 3-0 in Liga 1 Apertura 2024 February 4, 2024 // News Team  duel being between Sporting Cristal and Sport Boys. Both teams have a rich history and aim to be protagonists in League  in defense for Cristal.  Sporting Cristal is leading 3-0 against Sport Boys at the end of the first half. Cazonatti scored a goal for SportingFebruary 4, 2024 // News Team Sporting Cristal is leading 3-0 against Sport Boys at the end of the first half. Cazonatti scored a goal for Sporting Cristal, establishing their lead in Callao. Quispe almost made a mistake by failing to catch a cross from Ignácio. Sport Boys are struggling to put together passes and create offensive plays.'},
 {'url': 'https://www.vsstats.com/football/2024-02-04/sport-boys-vs-sporting-cristal',
  'content': 'Sport Boys VS Sporting Cristal Team Stats

### Setup the Retrieval part of the tool

Recall that retrievers need:
1. Source text
2. Document loader
3. Text splitter
4. Embedding model
5. Vector store
6. Actual retriever

In [18]:
loader = Docx2txtLoader("https://github.com/OHDSI/ETL-HealthVerityBuilder/raw/master/docs/WebMD_PBM_ETL_5.0.1_20170606.docx")
docs = loader.load()

In [19]:
docs

[Document(page_content='Source Data Mapping Approach to CDMV5.0.1\n\n\n\n\nTable name: stem_table\n\nReading from sample_medical_claims_20170502.csv\n\nWe pull all data into the STEM and we then allow the Vocabulary to decide which CDM table the data lands in.\n\n\n\nEXTRA COLUMNS FOR STEM\n\n========================\n\nIn addition, create new column STEM_ID as DATA_FEED+"-"+CLAIM_ID+"-"+RECORD_ID to act as a lookup for VISIT_OCCURRENCE_ID.\n\nWe can also leverage DATA_FEED, CLAIM_ID and RECORD_ID to help with sorting.\n\n\n\nDestination Field\n\nSource Field\n\nLogic\n\nComment\n\ndomain_id\n\n\n\n\n\n=====MEDICAL_CLAIMS=====\n\n========================\n\n\n\nDIAGNOSIS_CODE:\n\n================\n\nDefault domain = CONDITION from unless udpated by a Vocabulary mapping from CONCEPT_ID\n\n\n\nPROCEDURE_CODE:\n\n================\n\nDefault domain = PROCEDURE from unless updated by a Vocabulary mapping from CONCEPT_ID\n\n\n\n=====PHARMACY_CLAIMS=====\n\n==========================\n\nDefau